In [88]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.autograd import Variable

In [89]:
im_size = 224

In [90]:
data_dir = 'my_train'
def load_split_train_test(datadir, valid_size = .01):
    train_transforms = transforms.Compose([transforms.Resize([im_size,im_size]),
                                       transforms.ToTensor(),
                                       ])
    test_transforms = transforms.Compose([transforms.Resize([im_size,im_size]),
                                      transforms.ToTensor(),
                                      ])
    train_data = datasets.ImageFolder(datadir,       
                    transform=train_transforms)
    test_data = datasets.ImageFolder(datadir,
                    transform=test_transforms)
    num_train = len(train_data)
    print(num_train)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    from torch.utils.data.sampler import SubsetRandomSampler
    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    trainloader = torch.utils.data.DataLoader(train_data,
                   sampler=train_sampler, batch_size=32)
    testloader = torch.utils.data.DataLoader(test_data,
                   sampler=test_sampler, batch_size=32)
    return trainloader, testloader
trainloader, testloader = load_split_train_test(data_dir, .01)
# print(trainloader.dataset.)
print(trainloader.dataset.classes)

2314
['aircrafts', 'birds', 'cars', 'dogs', 'flowers']


In [91]:
device = torch.device("cuda" if torch.cuda.is_available() 
                                  else "cpu")
# model = models.inception_v3(pretrained=True)
model = models.vgg16(pretrained=True)
# model = models.alexnet(pretrained=True)
# model = models.resnet18(pretrained=True)
# model = models.densenet121(pretrained=True)
# model = models.resnet18(pretrained=True)
model.eval()
# print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [ ]:
# #for alaexnet
# for param in model.parameters():
#     param.requires_grad = False
    
# model.classifier = nn.Sequential(
#                                 nn.Linear(1024, 4096),
#                                  nn.ReLU(),
#                                  nn.Dropout(0.2),
#                                  nn.Linear(4096, 4096),
#                                  nn.ReLU(),
#                                  nn.Linear(4096, 5),
#                                  nn.LogSoftmax(dim=1))
# criterion = nn.NLLLoss()
# optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)
# model.to(device)

In [ ]:
# #for alaexnet
# for param in model.parameters():
#     param.requires_grad = False
    
# model.classifier = nn.Sequential(nn.Dropout(0.2),
#                                 nn.Linear(9216, 4096),
#                                  nn.ReLU(),
#                                  nn.Dropout(0.2),
#                                  nn.Linear(4096, 4096),
#                                  nn.ReLU(),
#                                  nn.Linear(4096, 5),
#                                  nn.LogSoftmax(dim=1))
# criterion = nn.NLLLoss()
# optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)
# model.to(device)

In [ ]:
# # for inceptionv3
# for param in model.parameters():
#     param.requires_grad = False

# # model.AuxLogits = None
# # model.fc = nn.Linear(2048, 5)
# model.aux_logits = False
# model.fc = nn.Sequential(nn.Linear(2048, 5),
#                                  nn.LogSoftmax(dim=1))
# criterion = nn.NLLLoss()
# optimizer = optim.Adam(model.fc.parameters(), lr=0.003)
# model.to(device)

In [92]:
#for VGG16
# for param in model.parameters():
#     param.requires_grad = False

x = []

count = -1
for i in model.features.children():
    count+=1
    if count in [24, 25, 26, 27]:
        continue
    x.append(i)
    
# model.features = None
model.features = nn.Sequential(*x)
model.classifier = nn.Sequential(nn.Linear(25088, 5),
                                 nn.LogSoftmax(dim=1))
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [93]:
pytorch_total_params = sum(p.numel() for p in model.parameters())

In [94]:
pytorch_total_params

10120517

In [ ]:
# ct = 0
# for child in model.children():
#     ct += 1
#     if ct <= 6:
#         for param in child.parameters():
#             param.requires_grad = False


In [ ]:
# For ResNet18
# for param in model.parameters():
#     param.requires_grad = False
    
# model.fc = nn.Sequential(nn.Linear(512, 2048),
#                                  nn.ReLU(),
#                                  nn.Dropout(0.2),
#                                  nn.Linear(2048, 5),
#                                  nn.LogSoftmax(dim=1))
# # model.fc = nn.Sequential(nn.Linear(512, 5),
# #                                  nn.LogSoftmax(dim=1))
# criterion = nn.NLLLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)
# model.to(device)

In [95]:
epochs = 3
steps = 0
running_loss = 0
print_every = 10
train_losses, test_losses = [], []
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        inputs, labels = inputs.to(device),labels.to(device)
        optimizer.zero_grad()
#         print("shape : ", inputs[0].shape)
#         nlist = []
#         for i in range(64):
#             inputs[i] = inputs[i].view(inputs[i].size(0), -1)
#             nlist.append(p)
#         inputs = torch.FloatTensor(nlist)
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    test_loss += batch_loss.item()
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            train_losses.append(running_loss/len(trainloader))
            test_losses.append(test_loss/len(testloader))                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss : {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            acc = accuracy/len(testloader)
            if acc >= 0.99:
                torch.save(model, 'models/coarse_grained_vgg16_saved.pth')
            running_loss = 0
            model.train()
torch.save(model, 'models/coarse_grained_vgg16.pth')

Epoch 1/3.. Train loss : 0.893.. Test loss: 0.060.. Test accuracy: 1.000
Epoch 1/3.. Train loss : 0.239.. Test loss: 0.006.. Test accuracy: 1.000
Epoch 1/3.. Train loss : 0.201.. Test loss: 0.003.. Test accuracy: 1.000
Epoch 1/3.. Train loss : 0.179.. Test loss: 0.001.. Test accuracy: 1.000
Epoch 1/3.. Train loss : 0.135.. Test loss: 0.002.. Test accuracy: 1.000
Epoch 1/3.. Train loss : 0.074.. Test loss: 0.001.. Test accuracy: 1.000
Epoch 1/3.. Train loss : 0.103.. Test loss: 0.000.. Test accuracy: 1.000
Epoch 2/3.. Train loss : 0.024.. Test loss: 0.007.. Test accuracy: 1.000
Epoch 2/3.. Train loss : 0.016.. Test loss: 0.002.. Test accuracy: 1.000
Epoch 2/3.. Train loss : 0.018.. Test loss: 0.001.. Test accuracy: 1.000
Epoch 2/3.. Train loss : 0.002.. Test loss: 0.001.. Test accuracy: 1.000
Epoch 2/3.. Train loss : 0.002.. Test loss: 0.001.. Test accuracy: 1.000


KeyboardInterrupt: 

In [99]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=torch.load('models/coarse_grained_vgg16.pth')

In [100]:
test_transforms = transforms.Compose([transforms.Resize([im_size,im_size]),
                                      transforms.ToTensor()])

def predict_image(image):
    image_tensor = test_transforms(image).float()
    image_tensor = image_tensor.unsqueeze_(0)
    inputa = Variable(image_tensor)
    inputa = inputa.to(device)
    output = model(inputa)
    index = output.data.cpu().numpy().argmax()
#     print(index)
    return index

In [102]:
import glob
from PIL import Image

lab = ['aircrafts', 'birds', 'cars', 'dogs', 'flowers']

count = 0
total = 0
for category in os.listdir("coarse_test"):
    for i in os.listdir("coarse_test/" + category):
        image = Image.open("coarse_test/" + category + "/" + i)
        index = predict_image(image)
        res = category == lab[index]
#         print(index)
        if res:
            count+=1
        total+=1
print("Test accuracy : ", count/total)
print(total)

Test accuracy :  0.9908256880733946
109


In [98]:
torch.save(model, 'models/coarse_grained_vgg16.pth')